In [9]:
import pandas as pd
import networkx as nx
import altair as alt
import numpy as np
import gzip
import urllib.request
import matplotlib.pyplot as plt

In [6]:
# Download and load the gzipped file
url = "https://snap.stanford.edu/data/facebook_combined.txt.gz"
filename = "facebook_combined.txt.gz"

In [11]:
# Download if not already present
urllib.request.urlretrieve(url, filename)

('facebook_combined.txt.gz', <http.client.HTTPMessage at 0x12a4b3b10>)

In [12]:
# Read the graph using gzip
with gzip.open(filename, 'rb') as f:
    G = nx.read_edgelist(f)

In [13]:
# Create DataFrame of node degrees
degree_dict = dict(G.degree())
nx.set_node_attributes(G, degree_dict, "degree")

In [14]:
node_data = pd.DataFrame({
    'node': list(degree_dict.keys()),
    'degree': list(degree_dict.values())
})

In [15]:
# Create Histogram of Node degree
alt.Chart(node_data).mark_bar().encode(
    x=alt.X("degree:Q", bin=alt.Bin(maxbins=30), title="Number of Friends"),
    y=alt.Y("count()", title="Number of Users"),
    tooltip=["count()"]
).properties(
    title="Distribution of Facebook User Friend Counts"
).interactive()

alt.Chart(...)

In [16]:
# Create scatterplot – Node Degree vs. Betweenness Centrality
centrality = nx.betweenness_centrality(G)
node_data["centrality"] = node_data["node"].map(centrality)

alt.Chart(node_data).mark_circle().encode(
    x=alt.X("degree:Q", title="Number of Friends"),
    y=alt.Y("centrality:Q", title="Betweenness Centrality"),
    color=alt.Color("centrality:Q", scale=alt.Scale(scheme='plasma')),
    tooltip=["node", "degree", "centrality"]
).properties(
    title="Node Centrality vs. Degree in Facebook Network"
).interactive()

alt.Chart(...)

In [18]:
# Add Interactivity
# Create slider parameter
slider = alt.param(
    name='degree_slider',
    bind=alt.binding_range(min=1, max=100, step=1, name='Max Degree:'),
    value=50  # initial value
)

In [19]:
# Add slider to the chart
chart = alt.Chart(node_data).mark_circle().encode(
    x=alt.X("degree:Q", title="Number of Friends"),
    y=alt.Y("centrality:Q", title="Betweenness Centrality"),
    color=alt.Color("centrality:Q", scale=alt.Scale(scheme='plasma')),
    tooltip=["node", "degree", "centrality"]
).transform_filter(
    alt.datum.degree <= slider
).add_params(
    slider
).properties(
    title="Interactive Centrality vs. Degree Plot"
)

chart

alt.Chart(...)